Twitter API
=========================================



You don't have to understand this code. 

In [1]:
# !pip install tweepy
# install twitter module for python

In [2]:
import os 
from pprint import pprint
import json
import tweepy
import numpy as np
import pandas as pd

In [3]:
#api keys
CONSUMER_KEY = "SYwFy7GtvskvMbHrTHo1vztoF"
CONSUMER_SECRET = "Czwdyk7kuIQqXNilYq5ZaiVYajVFfFPwSUCSSX1P9UielICQSV"
ACCESS_TOKEN = "1443610243972284424-idoK7JG7SK5IQiej8AtQ6gIOM9WaH4"
ACCESS_TOKEN_SECRET = "sQBnwwVcknqrGTbBz7H3aGXkD7kFX70z1sanSD8MJfWC3"


auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth,wait_on_rate_limit=True)

In [4]:
county_data = pd.read_csv("county_politics.csv")

In [5]:
dems = county_data[county_data["majority"]=="Democrat"]
dem_short = dems.sort_values("dem_pct", ascending= False).head() #find 5 highest proportion democrat voting counties
dem_short

,county,dem_pct,rep_pct,majority,lat,long
37,San Francisco,85.26%,12.72%,Democrat,37.779026,-122.419906
20,Marin,82.33%,15.79%,Democrat,38.040914,-122.619964
0,Alameda,79.83%,17.62%,Democrat,37.609029,-121.899142
43,Santa Cruz,78.44%,18.49%,Democrat,37.050096,-121.990591
40,San Mateo,77.89%,20.20%,Democrat,37.496904,-122.333057


In [6]:
reps = county_data[county_data["majority"]=="Republican"]
rep_short = reps.sort_values("rep_pct", ascending= False).head() #find 5 highest proportion democrat voting counties
rep_short

,county,dem_pct,rep_pct,majority,lat,long
17,Lassen,23.24%,74.47%,Republican,40.768558,-120.730998
24,Modoc,26.33%,71.19%,Republican,41.545049,-120.743600
51,Tehama,31.02%,66.62%,Republican,40.125133,-122.201553
44,Shasta,32.28%,65.41%,Republican,40.796512,-121.997919
10,Glenn,35.36%,62.52%,Republican,39.591277,-122.377866


In [7]:
def get_covid_tweets(lat,long):
    tweet_list = []
    response = api.search_tweets("covid",
                                geocode = str(lat) + "," + str(long) + ",100km",
                                count = 50,
                                result_type="recent",
                                lang="en")
    for i in response: 
        tweet_list.append(i.text.split("https")[0])
    
    return tweet_list

# Republican County Tweets

In [8]:
rep_twt = []

for i in range (0,5):
    rep_twt += get_covid_tweets(rep_short.iloc[i,4],rep_short.iloc[i,5]) #search each county for covid tweets

In [9]:
rep_twt_df = pd.DataFrame(rep_twt, columns = ["tweet"]).drop_duplicates() #create dataframe for tweets, drops duplicates
# rep_twt_df.to_csv("republican_tweets",index = False)

In [10]:
rep_twt_df

,tweet
0,Dems set up clash with GOP by pairing Covid re...
1,"@covid_priest @christogrozev @mod_russia Oh, i..."
2,"Trucker convoy laps Washington, DC, beltway to..."
3,Biden’s ‘test to treat’ covid plan draws prais...
4,@Kenny_Wallace My kids dad for covid pneumonia...
...,...
194,19 New Cases of COVID-19 Identified in Mendoci...
195,19 New Cases of COVID-19 Identified in Mendoci...
196,19 New Cases of COVID-19 Identified in Mendoci...
198,All your Covid stimmies used for co-pays for m...


In [11]:
import text2emotion as te

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/farhat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/farhat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/farhat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
rep_no_dupes = list(rep_twt_df.tweet) #get republican tweets without duplicates in list form

In [13]:
len(rep_no_dupes)

113

In [14]:
rep_twt_master = " ".join(rep_no_dupes)
rep_emotion = te.get_emotion(rep_twt_master)
rep_emotion

{'Happy': 0.16, 'Angry': 0.04, 'Surprise': 0.22, 'Sad': 0.21, 'Fear': 0.36}

# Democrat County Tweets

In [15]:
dem_twt = []

#since all the top 5 democratic counties are within close proximity, 
#I choose to use only one location within 100km radius

response = api.search_tweets("covid", geocode = str(37.779026) + "," + str(-122.419906) + ",100km",
                    count = 200,
                    result_type="recent",
                    lang="en")

for i in response: 
    dem_twt.append(i.text.split("https")[0])


In [16]:
len(dem_twt)

100

In [17]:
dem_twt

["@ConversationEDU @DoctorKarl @UNSW @hollyseale It's trivial to remove COVID vaccines from your body:\n\nJust wait about 60 days.",
 "@TAniplaylist (2/2) Also, since you've recently filed an appeal, it's possible that we're still reviewing it. Note:… ",
 'It is possible to get COVID-19 again. Immunity from initial vaccines and recovering from COVID-19 fades over time,… ',
 '#AI mines CT, lab results to predict #COVID-19 severity ',
 '@goldenstate1680 @49ersBBQLover There\'s that new word again "long covid"',
 '@HeroDividend Awesome list but I’m afraid the way this world is heading we might not survive 10 yrs.\n\nFirst they sa… ',
 "@zachtratar What's wrong with wanting freedom? I don't think anti-maskers (2 full years into COVID ffs w/ record lo… ",
 'On the relation between COVID-19, mobility, and the stock market ',
 '@Cardinals11in11 @cov_Gretchen What do you mean by “he mysteriously cured covid”? \n\nDemand is up bc 1) covid restri… ',
 "@AslamKh76148197 (2/2) Also, since you've r

In [18]:
dem_twt_df = pd.DataFrame(dem_twt, columns = ["tweet"]).drop_duplicates() #create dataframe for tweets, drops duplicates
#dem_twt_df.to_csv("dem_tweets",index = False)

In [19]:
dem_no_dupes = list(dem_twt_df.tweet)
dem_twt_master = " ".join(dem_no_dupes)

In [20]:
len(dem_no_dupes)

100

In [21]:
dem_emotion = te.get_emotion(dem_twt_master)
dem_emotion

{'Happy': 0.1, 'Angry': 0.08, 'Surprise': 0.19, 'Sad': 0.21, 'Fear': 0.42}

In [22]:
dem_emo_df = pd.DataFrame([list(dem_emotion.keys()),
              list(dem_emotion.values())]).T

dem_emo_df["majority"] = "Democrat"

dem_emo_df.columns = ["Emotion","Value","Majority"]

dem_emo_df

,Emotion,Value,Majority
0,Happy,0.1,Democrat
1,Angry,0.08,Democrat
2,Surprise,0.19,Democrat
3,Sad,0.21,Democrat
4,Fear,0.42,Democrat


In [23]:
rep_emo_df = pd.DataFrame([list(rep_emotion.keys()),
              list(rep_emotion.values())]).T

rep_emo_df["majority"] = "Republican"

rep_emo_df.columns = ["Emotion","Value","Majority"]

rep_emo_df

,Emotion,Value,Majority
0,Happy,0.16,Republican
1,Angry,0.04,Republican
2,Surprise,0.22,Republican
3,Sad,0.21,Republican
4,Fear,0.36,Republican


In [24]:
all_emotes = pd.concat([dem_emo_df,rep_emo_df])
all_emotes

,Emotion,Value,Majority
0,Happy,0.1,Democrat
1,Angry,0.08,Democrat
2,Surprise,0.19,Democrat
3,Sad,0.21,Democrat
4,Fear,0.42,Democrat
0,Happy,0.16,Republican
1,Angry,0.04,Republican
2,Surprise,0.22,Republican
3,Sad,0.21,Republican
4,Fear,0.36,Republican


In [25]:
#all_emotes.to_csv("tweet_emotions.csv",index=False)

In [29]:
import plotly.express as px
fig = px.bar(all_emotes, x="Emotion", y="Value", color="Majority", barmode = "group",
            title = "Emotions of Tweets Regarding COVID, Democratic Counties (n=99) vs. Republican Counties(n=103)")

fig.show()